In [1]:
"""
Pipeline Overview:

1. Data Loading & Preprocessing:
   - Resize to 224x224
   - Apply CLAHE for contrast enhancement
   - Denoise and Sharpen
   - Normalize to [0,1]

2. Data Augmentation (for class balancing):
   - Random brightness adjustment
   - Horizontal flip
   - Random rotation (-20° to +20°)
   - Zoom (0.9x to 1.1x)

3. Dataset Splitting:
   - Stratified Train/Validation/Test split (70/15/15)

4. Model Architectures:
   - VGG16 (pretrained on ImageNet, frozen base)
   - ResNet50 (pretrained on ImageNet, frozen base)
   - GlobalAveragePooling2D
   - Dense(512) + Dropout(0.5)
   - Dense(2) Softmax output

5. Training:
   - Loss: SparseCategoricalCrossentropy
   - Optimizer: Adam
   - ReduceLROnPlateau callback (patience=2, factor=0.3)

6. Evaluation:
   - Accuracy, Precision, Recall, F1-Score
   - Confusion Matrix
   - ROC AUC Score and Curve
   - Loss & Accuracy plots
"""


import os
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [2]:
!git clone https://github.com/vrkavitha/violence_detection.git

Cloning into 'violence_detection'...
remote: Enumerating objects: 6005, done.
remote: Total 6005 (delta 0), reused 0 (delta 0), pack-reused 6005 (from 2)
Receiving objects: 100% (6005/6005), 322.58 MiB | 24.66 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Updating files: 100% (6041/6041), done.


In [3]:
DATA_DIR   = "/content/violence_detection/dataset"
CATEGORIES = ["violence", "non_violence"]
IMG_SIZE   = 224

In [4]:
def apply_clahe(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l,a,b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8,8))
    l_cl = clahe.apply(l)
    merged = cv2.merge([l_cl, a, b])
    return cv2.cvtColor(merged, cv2.COLOR_LAB2BGR)

In [5]:
def denoise_and_sharpen(image):
    den = cv2.fastNlMeansDenoisingColored(image, None, 7, 7, 7, 21)
    kernel = np.array([[0, -1, 0],
                       [-1, 5,-1],
                       [0, -1, 0]])
    sharp = cv2.filter2D(den, -1, kernel)
    return sharp

In [6]:
def random_augment(image):
    # Brightness
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV).astype(np.float32)
    hsv[:,:,2] = np.clip(hsv[:,:,2] * random.uniform(0.7,1.3), 0,255)
    image = cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2BGR)
    # Flip
    if random.random() < 0.5:
        image = cv2.flip(image, 1)
    # Rotate
    angle = random.uniform(-20, 20)
    M = cv2.getRotationMatrix2D((IMG_SIZE/2, IMG_SIZE/2), angle, 1.0)
    image = cv2.warpAffine(image, M, (IMG_SIZE, IMG_SIZE), borderMode=cv2.BORDER_REFLECT)
    # Zoom
    if random.random() < 0.5:
        zx = zy = random.uniform(0.9,1.1)
        image = cv2.resize(image, None, fx=zx, fy=zy)
        h,w = image.shape[:2]
        top = max((h-IMG_SIZE)//2,0)
        left= max((w-IMG_SIZE)//2,0)
        image = image[top:top+IMG_SIZE, left:left+IMG_SIZE]
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    return image

In [ ]:
from tqdm import tqdm

# Load + Preprocess
raw, clahe, denoise = {}, {}, {}
for cat in CATEGORIES:
    raw[cat] = []
    clahe[cat] = []
    denoise[cat] = []
    folder = os.path.join(DATA_DIR, cat)
    file_list = [f for f in os.listdir(folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    for fname in tqdm(file_list, desc=f'Processing {cat}', unit='img'):
        img = cv2.imread(os.path.join(folder, fname))
        if img is None:
            continue
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        raw[cat].append(img)
        c = apply_clahe(img)
        d = denoise_and_sharpen(c)
        clahe[cat].append(c)
        denoise[cat].append(d)

Processing violence:  11%|█         | 332/3034 [00:58<11:34,  3.89img/s]

In [ ]:
# Class counts
for cat in CATEGORIES:
    print(f"{cat}: {len(raw[cat])} images")

# Balance classes by augmenting smaller class
max_ct = max(len(raw[c]) for c in CATEGORIES)
augmented = {cat: [] for cat in CATEGORIES}
for cat in CATEGORIES:
    need = max_ct - len(denoise[cat])
    pool = denoise[cat]
    for _ in tqdm(range(need), desc=f"Augmenting {cat}"):
        aug_img = random_augment(random.choice(pool))
        augmented[cat].append(aug_img)

final_counts = {cat: len(denoise[cat]) + len(augmented[cat]) for cat in CATEGORIES}
print("Final counts after augmentation:")
for cat in CATEGORIES:
    print(f"{cat}: {final_counts[cat]}")

In [ ]:
# Pie chart
plt.figure(figsize=(6,6))
plt.pie(final_counts.values(), labels=final_counts.keys(), autopct="%1.1f%%")
plt.title("Class Distribution (After Balancing)")
plt.tight_layout()
plt.show()

In [ ]:
# Grid
def plot_big_grid(raw, clahe, denoise, categories, n=4):
    rows = len(categories)*n
    fig, axes = plt.subplots(rows, 3, figsize=(15, rows*3))
    idx = 0
    for cat in categories:
        picks = random.sample(range(len(raw[cat])), min(n, len(raw[cat])))
        for p in picks:
            axes[idx,0].imshow(cv2.cvtColor(raw[cat][p], cv2.COLOR_BGR2RGB))
            axes[idx,0].axis("off")
            axes[idx,0].set_title(f"{cat} Original")

            axes[idx,1].imshow(cv2.cvtColor(clahe[cat][p], cv2.COLOR_BGR2RGB))
            axes[idx,1].axis("off")
            axes[idx,1].set_title("CLAHE")

            axes[idx,2].imshow(cv2.cvtColor(denoise[cat][p], cv2.COLOR_BGR2RGB))
            axes[idx,2].axis("off")
            axes[idx,2].set_title("Denoise+Sharpen")

            idx += 1
    plt.tight_layout()
    plt.show()

plot_big_grid(raw, clahe, denoise, CATEGORIES, n=4)

In [ ]:
X = []
y = []

label_map = {cat: idx for idx, cat in enumerate(CATEGORIES)}

for cat in CATEGORIES:
    # Combine denoised + augmented images
    imgs = denoise[cat] + augmented[cat]
    X.extend(imgs)
    y.extend([label_map[cat]] * len(imgs))

X = np.array(X, dtype=np.float32)
y = np.array(y)

In [ ]:
# Normalize to [0,1]
X /= 255.0

print(f"dataset: {X.shape}, labels: {y.shape}")

In [ ]:
# First split train vs temp (train=70%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y)

# Split temp into val/test 50% each => 15% val, 15% test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp)

In [ ]:
print(f"Train: {X_train.shape[0]} samples")
print(f"Val:   {X_val.shape[0]} samples")
print(f"Test:  {X_test.shape[0]} samples")

In [ ]:
# Load base model
base_model = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# Freeze convolutional base
for layer in base_model.layers:
    layer.trainable = False

# Custom head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, verbose=1)

# Train
history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=20,
                    batch_size=32,
                    callbacks=[reduce_lr])


In [ ]:

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

# Classification report
print(classification_report(y_test, y_pred, target_names=CATEGORIES))

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Oranges',
            xticklabels=CATEGORIES, yticklabels=CATEGORIES)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# ROC-AUC
y_test_bin = tf.keras.utils.to_categorical(y_test, num_classes=2)
roc_auc = roc_auc_score(y_test_bin, y_pred_probs)
print(f"ROC AUC Score: {roc_auc:.4f}")

# ROC curve
fpr, tpr, _ = roc_curve(y_test_bin[:,1], y_pred_probs[:,1])
plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.4f}')
plt.plot([0,1], [0,1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Load base model
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# Freeze base layers
for layer in base_model.layers:
    layer.trainable = False

# Custom head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, verbose=1)

history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=20,
                    batch_size=32,
                    callbacks=[reduce_lr])


In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

print(classification_report(y_test, y_pred, target_names=CATEGORIES))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Purples',
            xticklabels=CATEGORIES, yticklabels=CATEGORIES)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
y_test_bin = tf.keras.utils.to_categorical(y_test, num_classes=2)
roc_auc = roc_auc_score(y_test_bin, y_pred_probs)
print(f"ROC AUC Score: {roc_auc:.4f}")

fpr, tpr, _ = roc_curve(y_test_bin[:,1], y_pred_probs[:,1])
plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.4f}')
plt.plot([0,1], [0,1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.grid()
plt.show()